# Battle of the Neighborhood - Los Angeles - Notebook

## Part 1: Importing and cleaning the data 
#### I found the neighborhood data with longitude and latitude from USC (University of Southern California) Neighborhood Data for Social Change. I will clean the data so that only the name, longitude and latitude are available.

In [113]:
# Importing Libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [114]:
!pip install folium==0.5.0

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [115]:
import folium

In [4]:
# The code was removed by Watson Studio for sharing.

,name,display_na,sqmi,type,latitude,longitude
0,Acton,Acton L.A. County Neighborhood (Current),39.339109,unincorporated-area,-118.169810,34.497355
1,Adams-Normandie,Adams-Normandie L.A. County Neighborhood (Curr...,0.805350,segment-of-a-city,-118.300208,34.031461
2,Agoura Hills,Agoura Hills L.A. County Neighborhood (Current),8.146760,standalone-city,-118.759884,34.146736
3,Agua Dulce,Agua Dulce L.A. County Neighborhood (Current),31.462632,unincorporated-area,-118.317104,34.504927
4,Alhambra,Alhambra L.A. County Neighborhood (Current),7.623814,standalone-city,-118.136512,34.085539


In [5]:
# cleaning the dataframe to only have name and long-lat

df = df_data_1.drop(['display_na', 'sqmi', 'type'], axis=1)
df.columns = ['Neighborhood', 'Longitude', 'Latitude']
df.head()

,Neighborhood,Longitude,Latitude
0,Acton,-118.169810,34.497355
1,Adams-Normandie,-118.300208,34.031461
2,Agoura Hills,-118.759884,34.146736
3,Agua Dulce,-118.317104,34.504927
4,Alhambra,-118.136512,34.085539


In [6]:
df.shape

(272, 3)

In [7]:
df.head()

,Neighborhood,Longitude,Latitude
0,Acton,-118.169810,34.497355
1,Adams-Normandie,-118.300208,34.031461
2,Agoura Hills,-118.759884,34.146736
3,Agua Dulce,-118.317104,34.504927
4,Alhambra,-118.136512,34.085539


## Part 2: Exploring Data 

### Exploring explore Los Angeles

In [8]:
address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="la_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Los Angeles are 34.0536909, -118.242766.


In [116]:
# create map of LA using latitude and longitude values
map_la = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(Neighborhood)
    #label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_la)  
    
map_la

In [10]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: FAPESK432Y3TVBAB5SPW3MZNBLYSZR0R52VOKSK4NLKVQ2AF
CLIENT_SECRET:4VIV2A250Z4J0TILHZHCOUJKTJZ012VOK02BCDXIIRKXO00M


In [11]:
# exploring the Santa Monica neighborhood

df.loc[200, 'Neighborhood']

'Santa Monica'

In [12]:
neighborhood_latitude = df.loc[200, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[200, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[200, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Santa Monica are 34.0218604991244, -118.48056671004801.


In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=FAPESK432Y3TVBAB5SPW3MZNBLYSZR0R52VOKSK4NLKVQ2AF&client_secret=4VIV2A250Z4J0TILHZHCOUJKTJZ012VOK02BCDXIIRKXO00M&v=20180605&ll=34.0218604991244,-118.48056671004801&radius=500&limit=100'

In [14]:
results = requests.get(url).json()

In [15]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Tacos Por Favor,Taco Place,34.019741,-118.480247
1,Memorial Park,Park,34.021195,-118.480436
2,Memorial Park Tennis Courts,Tennis Court,34.021171,-118.481498
3,The Cove Skatepark,Skate Park,34.020673,-118.480594
4,Co-Opportunity,Supermarket,34.024289,-118.482684


In [17]:
nearby_venues.shape

(12, 4)

### Let's recreate this with all the neighborhoods in LA

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
la_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Acton
Adams-Normandie
Agoura Hills
Agua Dulce
Alhambra
Alondra Park
Artesia
Altadena
Angeles Crest
Arcadia
Arleta
Arlington Heights
Athens
Atwater Village
Avalon
Avocado Heights
Azusa
Vermont-Slauson
Baldwin Hills/Crenshaw
Baldwin Park
Bel-Air
Bellflower
Bell Gardens
Green Valley
Bell
Beverly Crest
Beverly Grove
Burbank
Koreatown
Beverly Hills
Beverlywood
Boyle Heights
Bradbury
Brentwood
Broadway-Manchester
Calabasas
Canoga Park
Carson
Carthay
Castaic Canyons
Chatsworth
Castaic
Central-Alameda
Century City
Cerritos
Charter Oak
Chatsworth Reservoir
Chesterfield Square
Cheviot Hills
Chinatown
Citrus
Claremont
Northridge
Commerce
Compton
Cypress Park
La Mirada
Covina
Cudahy
Culver City
Del Aire
Del Rey
Desert View Highlands
Diamond Bar
Downey
Downtown
Duarte
Eagle Rock
East Compton
East Hollywood
East La Mirada
Elizabeth Lake
East Los Angeles
East Pasadena
East San Gabriel
Echo Park
El Monte
El Segundo
El Sereno
Elysian Park
Elysian Valley
Vermont Square
Encino
Exposition Park
Fairfax
Flo

In [20]:
print(la_venues.shape)
la_venues.head()

(2999, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Acton,34.497355,-118.169810,Epik Engineering,34.498718,-118.168046,Construction & Landscaping
1,Acton,34.497355,-118.169810,Alma Gardening Co.,34.494762,-118.172550,Construction & Landscaping
2,Adams-Normandie,34.031461,-118.300208,Orange Door Sushi,34.032485,-118.299368,Sushi Restaurant
3,Adams-Normandie,34.031461,-118.300208,7-Eleven,34.033027,-118.299960,Convenience Store
4,Adams-Normandie,34.031461,-118.300208,Shell,34.033095,-118.300025,Gas Station


In [21]:
# number of venues by the neighborhood in la
la_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Acton,2,2,2,2,2,2
Adams-Normandie,12,12,12,12,12,12
Agoura Hills,30,30,30,30,30,30
Agua Dulce,1,1,1,1,1,1
Alhambra,14,14,14,14,14,14
Alondra Park,1,1,1,1,1,1
Altadena,4,4,4,4,4,4
Arcadia,24,24,24,24,24,24
Arleta,5,5,5,5,5,5


In [22]:
print('There are {} uniques categories.'.format(len(la_venues['Venue Category'].unique())))

There are 325 uniques categories.


### Analyzing Each Neighborhood

In [23]:
# one hot encoding
la_onehot = pd.get_dummies(la_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
la_onehot['Neighborhood'] = la_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [la_onehot.columns[-1]] + list(la_onehot.columns[:-1])
la_onehot = la_onehot[fixed_columns]

la_onehot.head()

,Yoga Studio,ATM,Accessories Store,Airport,American Restaurant,Amphitheater,Andhra Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Check Cashing Service,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Residence Hall,College Theater,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Notary,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Gym,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish 

In [26]:
la_onehot.shape

(2999, 325)

In [117]:
la_grouped = la_onehot.groupby('Neighborhood').mean().reset_index()
la_grouped.head()

,Neighborhood,Yoga Studio,ATM,Accessories Store,Airport,American Restaurant,Amphitheater,Andhra Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Check Cashing Service,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Residence Hall,College Theater,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Notary,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Gym,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Rock Club,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish 

In [28]:
la_grouped.shape

(234, 325)

### Turn the top 10 into a pandas dataframe

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = la_grouped['Neighborhood']

for ind in np.arange(la_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(la_grouped.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(234, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,Construction & Landscaping,Women's Store,Filipino Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
1,Adams-Normandie,Sushi Restaurant,Furniture / Home Store,Latin American Restaurant,Dessert Shop,Convenience Store,Playground,Gas Station,Park,Taco Place,Grocery Store
2,Agoura Hills,Fast Food Restaurant,Gas Station,Breakfast Spot,Chinese Restaurant,Thai Restaurant,Bakery,BBQ Joint,Automotive Shop,Restaurant,Lounge
3,Agua Dulce,Airport,Women's Store,Film Studio,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
4,Alhambra,Convenience Store,Mexican Restaurant,Hardware Store,Pizza Place,Construction & Landscaping,Fast Food Restaurant,Video Store,Breakfast Spot,Home Service,Sporting Goods Shop


## Now let's cluster the neighborhood

In [64]:
# set number of clusters
kclusters = 10

la_grouped_clustering = la_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(la_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([0, 1, 1, 6, 8, 5, 1, 1, 1, 1, 1, 1, 8, 8, 1, 2, 8, 1, 8, 1],
      dtype=int32)

In [63]:
# add clustering labels to the data frame
#neighborhoods_venues_sorted.replace(0, 'Cluster Labels', kmeans.labels_)

la_merged = df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
la_merged = la_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

la_merged.dropna(subset=["Cluster Labels"], inplace =True) #uncomment this next time you run

la_merged.head(20)

,Neighborhood,Longitude,Latitude,Cluster Labels,Cluster-Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,-118.169810,34.497355,0.0,0.0,Construction & Landscaping,Women's Store,Filipino Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
1,Adams-Normandie,-118.300208,34.031461,1.0,1.0,Sushi Restaurant,Furniture / Home Store,Latin American Restaurant,Dessert Shop,Convenience Store,Playground,Gas Station,Park,Taco Place,Grocery Store
2,Agoura Hills,-118.759884,34.146736,1.0,1.0,Fast Food Restaurant,Gas Station,Breakfast Spot,Chinese Restaurant,Thai Restaurant,Bakery,BBQ Joint,Automotive Shop,Restaurant,Lounge
3,Agua Dulce,-118.317104,34.504927,6.0,6.0,Airport,Women's Store,Film Studio,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
4,Alhambra,-118.136512,34.085539,8.0,8.0,Convenience Store,Mexican Restaurant,Hardware Store,Pizza Place,Construction & Landscaping,Fast Food Restaurant,Video Store,Breakfast Spot,Home Service,Sporting Goods Shop
5,Alondra Park,-118.335156,33.889617,5.0,5.0,Park,Food Service,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
6,Artesia,-118.080101,33.866896,1.0,1.0,Indian Restaurant,Chinese Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Pizza Place,Salon / Barbershop,BBQ Joint,Gift Shop,Tea Room,Taiwanese Restaurant
7,Altadena,-118.136239,34.193871,1.0,1.0,Campground,Home Service,Pharmacy,Notary,Women's Store,Farmers Market,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
9,Arcadia,-118.030419,34.133230,1.0,1.0,Tea Room,Gym,Cosmetics Shop,Ice Cream Shop,Fast Food Restaurant,Bubble Tea Shop,Shopping Mall,Chinese Restaurant,Coffee Shop,Convenience Store
10,Arleta,-118.430757,34.243100,1.0,1.0,Bakery,Convenience Store,Construction & Landscaping,Video Store,Historic Site,Fast Food Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop


In [59]:
la_merged_2 = la_merged.dropna(subset = ["Cluster-Labels"])
la_merged_2['Cluster Labels'] = la_merged['Cluster Labels'].astype(int)
la_merged_2.head()

,Neighborhood,Longitude,Latitude,Cluster Labels,Cluster-Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,-118.169810,34.497355,0,0.0,Construction & Landscaping,Women's Store,Filipino Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
1,Adams-Normandie,-118.300208,34.031461,1,1.0,Sushi Restaurant,Furniture / Home Store,Latin American Restaurant,Dessert Shop,Convenience Store,Playground,Gas Station,Park,Taco Place,Grocery Store
2,Agoura Hills,-118.759884,34.146736,1,1.0,Fast Food Restaurant,Gas Station,Breakfast Spot,Chinese Restaurant,Thai Restaurant,Bakery,BBQ Joint,Automotive Shop,Restaurant,Lounge
3,Agua Dulce,-118.317104,34.504927,6,6.0,Airport,Women's Store,Film Studio,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
4,Alhambra,-118.136512,34.085539,8,8.0,Convenience Store,Mexican Restaurant,Hardware Store,Pizza Place,Construction & Landscaping,Fast Food Restaurant,Video Store,Breakfast Spot,Home Service,Sporting Goods Shop


In [71]:
la_merged_2.drop(['Cluster-Labels'], axis=1, inplace=True) # uncomment this next time you run
la_merged_2.head()

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,-118.169810,34.497355,0,Construction & Landscaping,Women's Store,Filipino Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
1,Adams-Normandie,-118.300208,34.031461,1,Sushi Restaurant,Furniture / Home Store,Latin American Restaurant,Dessert Shop,Convenience Store,Playground,Gas Station,Park,Taco Place,Grocery Store
2,Agoura Hills,-118.759884,34.146736,1,Fast Food Restaurant,Gas Station,Breakfast Spot,Chinese Restaurant,Thai Restaurant,Bakery,BBQ Joint,Automotive Shop,Restaurant,Lounge
3,Agua Dulce,-118.317104,34.504927,6,Airport,Women's Store,Film Studio,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
4,Alhambra,-118.136512,34.085539,8,Convenience Store,Mexican Restaurant,Hardware Store,Pizza Place,Construction & Landscaping,Fast Food Restaurant,Video Store,Breakfast Spot,Home Service,Sporting Goods Shop


In [118]:
# Visualizing Clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_merged_2['Latitude'], la_merged_2['Longitude'], la_merged_2['Neighborhood'], la_merged_2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Analyzing and Sorting Clusters for Users

### Which Neighborhoods are the best for each Venue Category

In [94]:
df_venue = la_merged_2[['Neighborhood', 'Cluster Labels', '1st Most Common Venue']]
df_venue.head()

,Neighborhood,Cluster Labels,1st Most Common Venue
0,Acton,0,Construction & Landscaping
1,Adams-Normandie,1,Sushi Restaurant
2,Agoura Hills,1,Fast Food Restaurant
3,Agua Dulce,6,Airport
4,Alhambra,8,Convenience Store


In [99]:
# The number of unique most common venues
len(df_venue['1st Most Common Venue'].unique())

101

In [109]:
df2 = df_venue.set_index('1st Most Common Venue')
df2.sort_values(by=['1st Most Common Venue'], inplace=True)
print(df2.shape)

(234, 2)


In [120]:
df2.head(10)

,Neighborhood,Cluster Labels
1st Most Common Venue,,
ATM,Vermont Knolls,1
Airport,Agua Dulce,6
American Restaurant,Encino,1
Arts & Crafts Store,Downtown,1
Automotive Shop,Irwindale,1
Automotive Shop,Panorama City,8
Automotive Shop,Manhattan Beach,1
Bagel Shop,Calabasas,1
Bakery,Cypress Park,8


In [121]:
la_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Acton,34.497355,-118.169810,Epik Engineering,34.498718,-118.168046,Construction & Landscaping
1,Acton,34.497355,-118.169810,Alma Gardening Co.,34.494762,-118.172550,Construction & Landscaping
2,Adams-Normandie,34.031461,-118.300208,Orange Door Sushi,34.032485,-118.299368,Sushi Restaurant
3,Adams-Normandie,34.031461,-118.300208,7-Eleven,34.033027,-118.299960,Convenience Store
4,Adams-Normandie,34.031461,-118.300208,Shell,34.033095,-118.300025,Gas Station
5,Adams-Normandie,34.031461,-118.300208,Little Xian,34.032292,-118.299465,Sushi Restaurant
6,Adams-Normandie,34.031461,-118.300208,Sushi Delight,34.032501,-118.299454,Sushi Restaurant
7,Adams-Normandie,34.031461,-118.300208,Tacos La Estrella,34.032230,-118.300757,Taco Place
8,Adams-Normandie,34.031461,-118.300208,El Rincon Hondureño,34.032527,-118.298860,Latin American Restaurant
9,Adams-Normandie,34.031461,-118.300208,Loren Miller Recreational Park,34.031335,-118.303717,Playground


### Seeing the Neighborhoods in Each Cluster

In [82]:
# Cluster1
la_merged_2.loc[la_merged_2['Cluster Labels'] == 0, la_merged_2.columns[[0] + list(range(4, la_merged_2.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,Construction & Landscaping,Women's Store,Filipino Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
73,East Pasadena,Business Service,Construction & Landscaping,Film Studio,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
94,Granada Hills,Construction & Landscaping,Women's Store,Filipino Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
104,Hasley Canyon,Construction & Landscaping,Women's Store,Filipino Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
128,Lake Los Angeles,Construction & Landscaping,Women's Store,Filipino Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


In [123]:
# Cluster2
la_merged_2.loc[la_merged_2['Cluster Labels'] == 1, la_merged_2.columns[[0] + list(range(4, la_merged_2.shape[1]))]].head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Adams-Normandie,Sushi Restaurant,Furniture / Home Store,Latin American Restaurant,Dessert Shop,Convenience Store,Playground,Gas Station,Park,Taco Place,Grocery Store
2,Agoura Hills,Fast Food Restaurant,Gas Station,Breakfast Spot,Chinese Restaurant,Thai Restaurant,Bakery,BBQ Joint,Automotive Shop,Restaurant,Lounge
6,Artesia,Indian Restaurant,Chinese Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Pizza Place,Salon / Barbershop,BBQ Joint,Gift Shop,Tea Room,Taiwanese Restaurant
7,Altadena,Campground,Home Service,Pharmacy,Notary,Women's Store,Farmers Market,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
9,Arcadia,Tea Room,Gym,Cosmetics Shop,Ice Cream Shop,Fast Food Restaurant,Bubble Tea Shop,Shopping Mall,Chinese Restaurant,Coffee Shop,Convenience Store
10,Arleta,Bakery,Convenience Store,Construction & Landscaping,Video Store,Historic Site,Fast Food Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop
11,Arlington Heights,Karaoke Bar,Vegetarian / Vegan Restaurant,Korean Restaurant,Intersection,Latin American Restaurant,Liquor Store,Chinese Restaurant,Taco Place,Sushi Restaurant,Bakery
13,Atwater Village,Taco Place,Ice Cream Shop,Grocery Store,Mobile Phone Shop,Food Court,Mexican Restaurant,Mediterranean Restaurant,Fast Food Restaurant,Shopping Mall,Fried Chicken Joint
16,Azusa,Thrift / Vintage Store,Liquor Store,Pizza Place,Coffee Shop,Big Box Store,Pharmacy,Fish & Chips Shop,Farmers Market,Escape Room,Ethiopian Restaurant
17,Vermont-Slauson,Check Cashing Service,Mobile Phone Shop,Burger Joint,Grocery Store,Sandwich Place,Clothing Store,Farm,English Restaurant,Escape Room,Ethiopian Restaurant


In [84]:
# Cluster3
la_merged_2.loc[la_merged_2['Cluster Labels'] == 2, la_merged_2.columns[[0] + list(range(4, la_merged_2.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Baldwin Hills/Crenshaw,Park,Clothing Store,Historic Site,Farmers Market,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
29,Beverly Hills,Sake Bar,Food Truck,Women's Store,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
30,Beverlywood,Food Truck,Park,Boutique,Historic Site,Business Service,Women's Store,Fast Food Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop
44,Cerritos,Food Truck,Korean Restaurant,Fast Food Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
45,Charter Oak,Park,Moving Target,American Restaurant,Home Service,Fast Food Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
96,Hacienda Heights,Park,Scenic Lookout,Concert Hall,Fast Food Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm
115,Huntington Park,Park,Burger Joint,Fried Chicken Joint,Women's Store,Fast Food Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm
155,Mission Hills,Park,Church,Travel & Transport,Basketball Court,Women's Store,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
160,Montecito Heights,Park,Lake,Women's Store,Fast Food Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm
177,Paramount,Indie Movie Theater,Park,Flea Market,Fast Food Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm


In [85]:
# Cluster4
la_merged_2.loc[la_merged_2['Cluster Labels'] == 3, la_merged_2.columns[[0] + list(range(4, la_merged_2.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
222,South Whittier,Basketball Court,Women's Store,English Restaurant,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant


In [86]:
# Cluster5
la_merged_2.loc[la_merged_2['Cluster Labels'] == 4, la_merged_2.columns[[0] + list(range(4, la_merged_2.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Glendale,Trail,Pharmacy,Women's Store,Farmers Market,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm
161,Mount Washington,Trail,Park,Garden,Women's Store,Farmers Market,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
191,Rancho Palos Verdes,Trail,Women's Store,Fast Food Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
211,Sierra Madre,Trail,Gourmet Shop,Coffee Shop,Women's Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
235,Tujunga,Trail,Lake,Women's Store,Fast Food Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm
241,Unincorporated Santa Monica Mountains,Trail,Women's Store,Fast Food Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market


In [87]:
# Cluster6
la_merged_2.loc[la_merged_2['Cluster Labels'] == 5, la_merged_2.columns[[0] + list(range(4, la_merged_2.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Alondra Park,Park,Food Service,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
40,Chatsworth,Park,Food Service,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
102,Harvard Park,Park,Food Service,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
122,La Crescenta-Montrose,Park,Food Service,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
135,La Verne,Park,Food Service,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
218,South Park,Park,Food Service,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
248,Veterans Administration,Park,Food Service,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market


In [89]:
# Cluster7
la_merged_2.loc[la_merged_2['Cluster Labels'] == 6, la_merged_2.columns[[0] + list(range(4, la_merged_2.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Agua Dulce,Airport,Women's Store,Film Studio,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


In [90]:
# Cluster8
la_merged_2.loc[la_merged_2['Cluster Labels'] == 7, la_merged_2.columns[[0] + list(range(4, la_merged_2.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Green Valley,Food,Summer Camp,Home Service,Women's Store,Fast Food Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
32,Bradbury,Home Service,Women's Store,Filipino Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
120,Industry,Racetrack,Home Service,Trail,Arts & Entertainment,Farmers Market,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
184,Porter Ranch,Home Service,Park,Electronics Store,Farmers Market,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
229,Temple City,Home Service,Bank,Pharmacy,Women's Store,Fast Food Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm
271,Woodland Hills,Home Service,Arts & Entertainment,Women's Store,Filipino Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market


In [122]:
# Cluster9
la_merged_2.loc[la_merged_2['Cluster Labels'] == 8, la_merged_2.columns[[0] + list(range(4, la_merged_2.shape[1]))]].head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Alhambra,Convenience Store,Mexican Restaurant,Hardware Store,Pizza Place,Construction & Landscaping,Fast Food Restaurant,Video Store,Breakfast Spot,Home Service,Sporting Goods Shop
14,Avalon,Golf Course,American Restaurant,Mexican Restaurant,Playground,Fast Food Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant
15,Avocado Heights,Mexican Restaurant,Fast Food Restaurant,Liquor Store,Diner,Pizza Place,Discount Store,Taco Place,Fried Chicken Joint,Bakery,Grocery Store
19,Baldwin Park,Italian Restaurant,Mexican Restaurant,Thrift / Vintage Store,Liquor Store,Business Service,Latin American Restaurant,Flea Market,Food,Food & Drink Shop,Escape Room
22,Bell Gardens,Mexican Restaurant,Burger Joint,Seafood Restaurant,Convenience Store,Men's Store,Park,Latin American Restaurant,Fried Chicken Joint,New American Restaurant,Fast Food Restaurant
36,Canoga Park,Mexican Restaurant,Ice Cream Shop,Restaurant,Sushi Restaurant,Sports Bar,Dessert Shop,Film Studio,Fabric Shop,Falafel Restaurant,Farm
42,Central-Alameda,Kitchen Supply Store,Mexican Restaurant,Donut Shop,Liquor Store,Women's Store,Fast Food Restaurant,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm
47,Chesterfield Square,Burger Joint,Grocery Store,Mexican Restaurant,Mediterranean Restaurant,Gift Shop,Hardware Store,Marijuana Dispensary,Women's Store,Farmers Market,Escape Room
50,Citrus,Japanese Restaurant,Food,Mexican Restaurant,Convenience Store,Restaurant,Donut Shop,Italian Restaurant,Bakery,Fast Food Restaurant,Escape Room
55,Cypress Park,Bakery,Park,Mexican Restaurant,Discount Store,Women's Store,Filipino Restaurant,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm


In [92]:
# Cluster10
la_merged_2.loc[la_merged_2['Cluster Labels'] == 9, la_merged_2.columns[[0] + list(range(4, la_merged_2.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
156,Monterey Park,Record Shop,Food Service,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
178,Pico Rivera,Record Shop,Dog Run,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
